In [ ]:
import pandas as pd
from river import (
    preprocessing,
    cluster,
    evaluate,
    stream,
    metrics
)
import os
import sys
sys.path.append('../')
from vibrodiagnostics import selection, models


PATH_PREFIX = '../../datasets/'
FEATURES_PATH =  os.path.join(PATH_PREFIX, 'features_data')

DATASET_PATH = os.path.join(PATH_PREFIX, 'MAFAULDA.zip')
MAFAULDA_METADATA = os.path.join(FEATURES_PATH, 'mafaulda_metadata.csv')

TD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_FEATURES_PATH)
FD_FEATURES = os.path.join(FEATURES_PATH, selection.FREQ_FEATURES_PATH)
TD_FD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_AND_FREQ_FEATURES_PATH)

TSFEL_FEATURES = os.path.join(FEATURES_PATH, selection.TSFEL_FEATURES_PATH)
TSFEL_TD_FEATURES = os.path.join(FEATURES_PATH, selection.TSFEL_TIME_FEATURES_PATH)
TSFEL_FD_FEATURES = os.path.join(FEATURES_PATH, selection.TSFEL_FREQ_FEATURES_PATH)
TSFEL_SD_FEATURES = os.path.join(FEATURES_PATH, selection.TSFEL_STAT_FEATURES_PATH)

In [ ]:
X, y = models.features_subset(TSFEL_FEATURES, selection.FAULT_CLASSES, ['az'], 'fault')
clustering = cluster.DenStream(
    decaying_factor=0.01, beta=0.5, mu=2.5, epsilon=0.5, n_samples_init=10
)
denstream = (
    preprocessing.StandardScaler() | clustering
)

silhouette = metrics.Silhouette()

for x, y in stream.iter_pandas(X, y):
    y_predict = denstream.predict_one(x)
    print([c.calc_center() for c in clustering.clusters.values()])
    #silhouette.update(x, y_predict)
    denstream.learn_one(x)

print(silhouette.get())

# Calculate offline silhoute at the end

#success = pd.DataFrame.from_records(steps).set_index('Step')
#success.plot(grid=True, figsize=(20, 5))

In [ ]:
X, y = models.features_subset(TSFEL_FEATURES, selection.FAULT_CLASSES, ['az'], 'fault')
denstream = (
    preprocessing.MinMaxScaler() |
    cluster.DenStream(
        decaying_factor=0.01, beta=0.5, mu=2.5, epsilon=0.5, n_samples_init=10
    )
)

predictions = []
for x, y in stream.iter_pandas(X, y):
    y_predict = denstream.predict_one(x)
    denstream.learn_one(x)              # denstream = denstream.learn_one(x)
    predictions.append({'y_true': y, 'y_predict': y_predict})


pd.DataFrame.from_records(predictions)